In [1]:
# Import all required packages
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,RandomizedSearchCV, GridSearchCV
from scipy.stats import randint,uniform
from sklearn.metrics import accuracy_score, roc_curve, roc_auc_score, PrecisionRecallDisplay, average_precision_score,precision_recall_curve,auc
import seaborn as sn
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree  import DecisionTreeClassifier
import warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from joblib import Parallel
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures,StandardScaler,MinMaxScaler
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV,train_test_split
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn import pipeline,set_config
from sklearn.utils import class_weight
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import accuracy_score, roc_curve, roc_auc_score, PrecisionRecallDisplay, precision_score, \
    average_precision_score, auc, precision_recall_curve, make_scorer,SCORERS
from skopt import BayesSearchCV
import datetime, re, sys, holidays
from Functions import *
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

set_config(display="diagram")
np.random.seed (0)
Parallel(n_jobs=8, max_nbytes=50000)

warnings.filterwarnings("ignore")

c:\users\rawwi\onedrive\school\nus\current sem\python\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
categorical_cols = ['Vessel Type']
numeric_cols = ['Waiting Time (Days)']
vessel_cols = ['Total Vessels (Number)', 'Capesize',
                'Capesize Laden', 'Capesize Unladen', 'Panamax', 'Panamax Laden',
                'Panamax Unladen', 'Handymax', 'Handymax Laden', 'Handymax Unladen',
                'Handysize', 'Handysize Laden', 'Handysize Unladen']
# date_cols = ['Berth or Port Entry', 'Date']
date_cols = ['Date']
dwt_cols = ['Total Vessels (DWT)','Capesize (DWT)',
            'Capesize Laden (DWT)', 'Capesize Unladen (DWT)', 'Panamax (DWT)',
            'Panamax Laden (DWT)', 'Panamax Unladen (DWT)', 'Handymax (DWT)',
            'Handymax Laden (DWT)', 'Handymax Unladen (DWT)', 'Handysize (DWT)',
            'Handysize Laden (DWT)', 'Handysize Unladen (DWT)']
weather_cols = ['tempt', 'prcp', 'wspd']

predict_col = 'Wait Levels'

In [3]:
# Get data
data = pd.read_csv('clean_data.csv', index_col=0, parse_dates=date_cols)
weather_df = pd.read_csv('sao_paulo_weather.csv', parse_dates=['date'])
weather_df = weather_df[['date','tavg','prcp','wspd']].rename(columns={'date': 'Date',
                   'tavg': 'tempt',
                   'prcp': 'prcp',
                   'wspd': 'wspd'}, inplace=False)
weather_df = weather_df.fillna(0)
data = data.merge(weather_df, how='inner',left_on=['Date'], right_on=['Date'])

# Drop cols with only 1 value
for col in data.columns:
    if len(data[col].unique()) == 1:
        data.drop(col,axis=1, inplace=True)

data.drop('Berth or Port Entry',axis=1, inplace=True)

#convert DWT cols from obj to int
for col in dwt_cols:
    data[col] = data[col].str.replace(",","").astype(int)
#convert vessel_cols from float to int
for col in vessel_cols:
    data[col] = data[col].astype(int)

#print cleaned data
data.head()

,IMO,Waiting Time (Days),Vessel Type,Date,Total Vessels (Number),Total Vessels (DWT),Capesize,Capesize Laden,Capesize Unladen,Panamax,...,Handymax (DWT),Handymax Laden (DWT),Handymax Unladen (DWT),Handysize (DWT),Handysize Laden (DWT),Handysize Unladen (DWT),Wait Levels,tempt,prcp,wspd
0,9056399,0.2,3,2015-01-02,16,878272,0,0,0,5,...,432704,121785,310919,52823,30060,22763,0,27.0,0.0,15.1
1,9471252,3.4,2,2015-01-02,16,878272,0,0,0,5,...,432704,121785,310919,52823,30060,22763,1,27.0,0.0,15.1
2,9233399,5.6,3,2015-01-02,16,878272,0,0,0,5,...,432704,121785,310919,52823,30060,22763,2,27.0,0.0,15.1
3,8309141,0.5,1,2015-01-03,17,908718,0,0,0,5,...,474533,107174,367359,70297,36036,34261,0,23.3,3.0,13.8
4,9625970,3.9,2,2015-01-03,17,908718,0,0,0,5,...,474533,107174,367359,70297,36036,34261,1,23.3,3.0,13.8


In [4]:
data.to_csv('data')

In [5]:
# split to train & Test
X_train, X_test, y_train, y_test = train_test_split(data.drop(predict_col,axis=1, inplace=False), data[predict_col], test_size=0.9, random_state=0)

Create Pipeline

In [6]:
date_linear_processor = pipeline.Pipeline([('ProcessDates',ProcessDates()),('ProcessHolidays',ProcessHolidays()),('DropDate',DropDate(date_cols))])
numeric_linear_processor = pipeline.Pipeline([('Polynomial',PolynomialFeatures(degree=2,interaction_only =True,include_bias=True)),('Scaler',StandardScaler())])
cat_linear_processor = OneHotEncoder()
vessel_linear_processor = ProcessVessel()
dwt_linear_processor = ProcessDWT()
weather_linear_processor = ProcessWeather()

data_preprocessor = ColumnTransformer([
    # ('numeric_linear_processor',numeric_linear_processor, numeric_cols),
    ('cat_linear_processor',cat_linear_processor, categorical_cols),
    ('date_linear_processor',date_linear_processor, date_cols),
    ('vessel_linear_processor',vessel_linear_processor, vessel_cols),
    ('dwt_linear_processor',dwt_linear_processor, dwt_cols),
    ('weather_linear_processor',weather_linear_processor, weather_cols)
])
data_preprocessor


ColumnTransformer(transformers=[('cat_linear_processor', OneHotEncoder(),
                                 ['Vessel Type']),
                                ('date_linear_processor',
                                 Pipeline(steps=[('ProcessDates',
                                                  ProcessDates()),
                                                 ('ProcessHolidays',
                                                  ProcessHolidays()),
                                                 ('DropDate',
                                                  DropDate(date_cols=['Date']))]),
                                 ['Date']),
                                ('vessel_linear_processor', ProcessVessel(),
                                 ['Total Vessels (Number)', 'Capesize',
                                  'Capesize Laden', 'Ca...
                                ('dwt_linear_processor', ProcessDWT(),
                                 ['Total Vessels (DWT)', 'Capesize (DWT)',
                                  'Capesize Laden (DWT)',
                                  'Capesize Unladen (DWT)', 'Panamax (DWT)',
                                  'Panamax Laden (DWT)',
                                  'Panamax Unladen (DWT)', 'Handymax (DWT)',
                                  'Handymax Laden (DWT)',
                                  'Handymax Unladen (DWT)', 'Handysize (DWT)',
                                  'Handysize Laden (DWT)',
                                  'Handysize Unladen (DWT)']),
                                ('weather_linear_processor', ProcessWeather(),
                                 ['tempt', 'prcp', 'wspd'])])

In [7]:
finalPipe = pipeline.make_pipeline(data_preprocessor)
# dp = finalPipe()
finalPipe.fit_transform(data)

array([[0.0, 0.0, 0.0, ..., 27.0, 0.0, 15.1],
       [0.0, 0.0, 1.0, ..., 27.0, 0.0, 15.1],
       [0.0, 0.0, 0.0, ..., 27.0, 0.0, 15.1],
       ...,
       [0.0, 0.0, 0.0, ..., 21.6, 0.0, 9.8],
       [0.0, 0.0, 1.0, ..., 20.3, 0.0, 10.6],
       [0.0, 0.0, 0.0, ..., 21.2, 0.0, 14.6]], dtype=object)

Regressor model

In [23]:
param_grid = {
    'columntransformer__numeric_linear_processor__Polynomial__degree': [1,2,3],
    'columntransformer__numeric_linear_processor__Polynomial__include_bias': [True,False],
    'columntransformer__numeric_linear_processor__Polynomial__interaction_only': [True,False],

    'stackingclassifier__LightGBM__num_leaves': range(100,1500,300),
    'stackingclassifier__LightGBM__n_estimators': range(100,1500,300),
    'stackingclassifier__LightGBM__reg_lambda': [ 0.11, 1, 10, 50, 100, 150],

    'stackingclassifier__XGBoost__reg_lambda': [0, 1e-1, 1, 10, 50],
    'stackingclassifier__XGBoost__n_estimators': range(100,1500,300),
    'stackingclassifier__XGBoost__eta': [0.1,0.5,1.0],

    'stackingclassifier__RandomForest__max_depth': [10, 30, 50],
    'stackingclassifier__RandomForest__max_features': [5,10,15],
    'stackingclassifier__RandomForest__min_samples_leaf': [3, 5, 7],

    'stackingclassifier__LinearRegression__alpha': [10, 30, 50],
    'stackingclassifier__LinearRegression__l1_ratio': [5,10,15],

    'stackingclassifier__final_estimator__num_leaves': range(100,1500,300),
    'stackingclassifier__final_estimator__n_estimators': range(100,1500,300),
    'stackingclassifier__final_estimator__reg_lambda': [ 0.11, 1, 10, 50, 100, 150],
    'stackingclassifier__final_estimator__learning_rate': [0.1,0.5,1.0]

}

In [40]:
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm  import LGBMRegressor
from sklearn.ensemble import StackingRegressor
estimators = [
    ("LinearRegression", ElasticNet( random_state=3)),
    ("GradientBoost", GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10,
                                   loss='huber', random_state =5)),
    ("XGBoost", XGBRegressor(objective='reg:squarederror',eval_metric='rmse',random_state=0,n_jobs=-1,use_label_encoder=False,  tree_method = 'gpu_hist')),
    ("RandomForest", RandomForestRegressor(random_state=0)),
    ("LGBM", LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)),
]
finalestimators = []
for estimator, model in estimators:
    finalestimators.append((estimator,pipeline.make_pipeline(data_preprocessor,model)))
final_estimator=LGBMRegressor(objective='regression',random_state=0,n_jobs=-1,device = 'gpu')
stacking_Regressor = StackingRegressor(estimators=finalestimators, final_estimator=final_estimator,n_jobs=-1)
# finalPipe = pipeline.make_pipeline(data_preprocessor,stacking_Regressor)
stacking_Regressor

StackingRegressor(estimators=[('LinearRegression',
                               Pipeline(steps=[('columntransformer',
                                                ColumnTransformer(transformers=[('cat_linear_processor',
                                                                                 OneHotEncoder(),
                                                                                 ['Vessel '
                                                                                  'Type']),
                                                                                ('date_linear_processor',
                                                                                 Pipeline(steps=[('ProcessDates',
                                                                                                  ProcessDates()),
                                                                                                 ('ProcessHolidays',
                                                                                                  ProcessHolidays()),
                                                                                                 ('DropDate',
                                                                                                  DropDate(date_cols=['Date']))]),
                                                                                 ['Date']),
                                                                                ('vessel...
                                               ('lgbmregressor',
                                                LGBMRegressor(bagging_fraction=0.8,
                                                              bagging_freq=5,
                                                              bagging_seed=9,
                                                              feature_fraction=0.2319,
                                                              feature_fraction_seed=9,
                                                              learning_rate=0.05,
                                                              max_bin=55,
                                                              min_data_in_leaf=6,
                                                              min_sum_hessian_in_leaf=11,
                                                              n_estimators=720,
                                                              num_leaves=5,
                                                              objective='regression'))]))],
                  final_estimator=LGBMRegressor(device='gpu',
                                                objective='regression',
                                                random_state=0),
                  n_jobs=-1)

In [41]:
stacking_Regressor.get_params()

{'cv': None,
 'estimators': [('LinearRegression',
   Pipeline(steps=[('columntransformer',
                    ColumnTransformer(transformers=[('cat_linear_processor',
                                                     OneHotEncoder(),
                                                     ['Vessel Type']),
                                                    ('date_linear_processor',
                                                     Pipeline(steps=[('ProcessDates',
                                                                      ProcessDates()),
                                                                     ('ProcessHolidays',
                                                                      ProcessHolidays()),
                                                                     ('DropDate',
                                                                      DropDate(date_cols=['Date']))]),
                                                     ['Date']),
            

In [45]:
#tune hyperparameter
limitTuner = 'LinearRegression'
preprocessingName = 'columntransformer__'
estimatorsParam = {}
finalParam = {}
for estimator in estimators:
    if limitTuner== '' or estimator[0] ==limitTuner:
        modelParam = {}
        for param, grid in param_grid.items():
            if param.startswith(estimator[0]):
                modelParam[param] = grid
            elif param.startswith(preprocessingName) :
                modelParam[estimator[0]+'__'+param] = grid
        print('Tuning Model ',estimator[0], "with params:\n",  modelParam, '\n')
        gs = GridSearchCV(estimator=estimator[1],param_grid=modelParam,scoring = 'rmse',n_jobs = -1,cv = 5,verbose = -1)
        gs.fit(X_train,X_test)
        print('Tuned Model ',estimator[0], " (RMSE = ","{:.4f}".format(gs.best_score_),")")
        # print("Best params:\n",  gs.best_params_, '\n')
        estimatorsParam ={**estimatorsParam, **gs.best_params_}

print('\n Params to use\n\n',estimatorsParam)

Tuning Model  LinearRegression with params:
 {'LinearRegression__columntransformer__numeric_linear_processor__Polynomial__degree': [1, 2, 3], 'LinearRegression__columntransformer__numeric_linear_processor__Polynomial__include_bias': [True, False], 'LinearRegression__columntransformer__numeric_linear_processor__Polynomial__interaction_only': [True, False]} 

Tuned Model  LinearRegression  (RMSE =  0.1245 )

 Params to use

 {'LinearRegression__max_depth': 4, 'LinearRegression__n_estimators': 60, 'columntransformer__numeric_linear_processor__Polynomial__degree': 3, 'columntransformer__numeric_linear_processor__Polynomial__include_bias': True, 'columntransformer__numeric_linear_processor__Polynomial__interaction_only': True}


In [29]:
# replace individual best model here post tuning.

# estimatorsParam = {}

In [47]:
# prepare data for final tuning
finalPipe.set_params(**estimatorsParam)
finalTrain  = finalPipe.fit_transform()
finalGrid = {}

# generate final model params
for param, grid in param_grid.items():
    if param.startswith("final_estimator"):
        finalGrid[param[len("final_estimator__"):]] = grid

#tune final model
gs = GridSearchCV(estimator=final_estimator,param_grid=finalGrid,scoring = 'rmse',n_jobs = -1,cv = 5,verbose = -1)
gs.fit(finalTrain)
modelParam = {}
for param, grid in gs.best_params_:
    modelParam["final_estimator__"+param] = grid
finalParam = {**estimatorsParam, **modelParam}

NameError: name 'finalTrain' is not defined

Old classifier model

estimators = [
    ("LightGBM", LGBMClassifier(objective='multiclass',random_state=0,n_jobs=-1,device = 'gpu',)),
    ("XGBoost", XGBClassifier(objective='multi:softmax',random_state=0,n_jobs=-1,use_label_encoder=False,  tree_method = 'gpu_hist')),
    ("RandomForest", RandomForestClassifier(random_state=0)),
    ("DecisionTree", DecisionTreeClassifier()),
]
final_estimator=LGBMClassifier(objective='multiclass',random_state=0,n_jobs=-1,device = 'gpu')
stacking_classifier = StackingClassifier(estimators=estimators, final_estimator=final_estimator,stack_method='predict_proba',n_jobs=-1)
finalPipe = pipeline.make_pipeline(data_preprocessor,stacking_classifier)
finalPipe

In [ ]:
finalPipe.get_params()

In [ ]:
param_grid = {
    'columntransformer__numeric_linear_processor__Polynomial__degree': [1,2,3],
    'columntransformer__numeric_linear_processor__Polynomial__include_bias': [True,False],
    'columntransformer__numeric_linear_processor__Polynomial__interaction_only': [True,False],

    'stackingclassifier__LightGBM__num_leaves': range(100,1500,300),
    'stackingclassifier__LightGBM__n_estimators': range(100,1500,300),
    'stackingclassifier__LightGBM__reg_lambda': [ 0.11, 1, 10, 50, 100, 150],

    'stackingclassifier__XGBoost__reg_lambda': [0, 1e-1, 1, 10, 50],
    'stackingclassifier__XGBoost__n_estimators': range(100,1500,300),
    'stackingclassifier__XGBoost__eta': [0.1,0.5,1.0],

    'stackingclassifier__RandomForest__max_depth': [10, 30, 50],
    'stackingclassifier__RandomForest__max_features': [5,10,15],
    'stackingclassifier__RandomForest__min_samples_leaf': [3, 5, 7],

    'stackingclassifier__DecisionTree__max_depth': [10, 30, 50],
    'stackingclassifier__DecisionTree__max_features': [5,10,15],
    'stackingclassifier__DecisionTree__min_samples_leaf': [3, 5, 7],

    # 'stackingclassifier__final_estimator__num_leaves': range(100,1500,300),
    # 'stackingclassifier__final_estimator__n_estimators': range(100,1500,300),
    # 'stackingclassifier__final_estimator__reg_lambda': [ 0.11, 1, 10, 50, 100, 150],
    # 'stackingclassifier__final_estimator__learning_rate': [0.1,0.5,1.0]

}
iter =50
rs = RandomizedSearchCV(
    estimator=finalPipe, param_distributions=param_grid,
    n_iter=iter,
    cv=5,
    refit=True,
    random_state=314,
    verbose=10,
    n_jobs=8,
    pre_dispatch=15)

rs.fit(X_train, y_train)

rs.best_estimator_

In [ ]:
rs.best_params_

In [ ]:
predict_test = rs.predict(X_test)
accuracy_test = accuracy_score(y_test,predict_test)
accuracy_test

In [ ]:
predict_test

In [ ]:
predict_prob_test = rs.predict_proba(X_test)
roc_auc_score(y_score=predict_prob_test, y_true=y_test, multi_class="ovr")